In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Шаг 1: Определите модели Pydantic для структурированных выводов

Эти модели определяют **схему**, которую агенты будут возвращать. Использование `response_format` с Pydantic обеспечивает:
- ✅ Безопасное извлечение данных с учетом типов
- ✅ Автоматическую проверку
- ✅ Отсутствие ошибок при разборе ответов в свободном тексте
- ✅ Удобную условную маршрутизацию на основе полей


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Шаг 2: Создание инструмента для бронирования отелей

Этот инструмент будет использовать **availability_agent**, чтобы проверять доступность номеров. Мы используем декоратор `@ai_function` для:
- Преобразования функции Python в инструмент, доступный для вызова ИИ
- Автоматической генерации JSON-схемы для LLM
- Проверки параметров
- Автоматического вызова агентами

Для этого примера:
- **Стокгольм, Сиэтл, Токио, Лондон, Амстердам** → Есть номера ✅
- **Все остальные города** → Нет номеров ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Шаг 3: Определение функций условий для маршрутизации

Эти функции анализируют ответ агента и определяют, какой путь выбрать в рабочем процессе.

**Основной шаблон:**
1. Проверить, является ли сообщение `AgentExecutorResponse`
2. Разобрать структурированный вывод (модель Pydantic)
3. Вернуть `True` или `False` для управления маршрутизацией

Рабочий процесс будет оценивать эти условия на **рёбрах**, чтобы решить, какой исполнитель будет вызван следующим.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Шаг 4: Создание пользовательского исполнителя отображения

Исполнители — это компоненты рабочего процесса, которые выполняют преобразования или побочные эффекты. Мы используем декоратор `@executor` для создания пользовательского исполнителя, который отображает конечный результат.

**Основные понятия:**
- `@executor(id="...")` - Регистрирует функцию как исполнитель рабочего процесса
- `WorkflowContext[Never, str]` - Подсказки типов для ввода/вывода
- `ctx.yield_output(...)` - Возвращает конечный результат рабочего процесса


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Шаг 5: Загрузка переменных окружения

Настройте клиент LLM. Этот пример работает с:
- **Моделями GitHub** (бесплатный уровень с токеном GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Шаг 6: Создание AI-агентов с структурированными выводами

Мы создаем **трех специализированных агентов**, каждый из которых обернут в `AgentExecutor`:

1. **availability_agent** - Проверяет доступность номеров в отеле с помощью инструмента
2. **alternative_agent** - Предлагает альтернативные города (если нет свободных номеров)
3. **booking_agent** - Побуждает к бронированию (если номера доступны)

**Основные особенности:**
- `tools=[hotel_booking]` - Предоставляет инструмент агенту
- `response_format=PydanticModel` - Обеспечивает структурированный вывод в формате JSON
- `AgentExecutor(..., id="...")` - Оборачивает агента для использования в рабочем процессе


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Шаг 7: Создание рабочего процесса с условными переходами

Теперь мы используем `WorkflowBuilder` для построения графа с условной маршрутизацией:

**Структура рабочего процесса:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Основные методы:**
- `.set_start_executor(...)` - Устанавливает точку входа
- `.add_edge(from, to, condition=...)` - Добавляет условный переход
- `.build()` - Завершает создание рабочего процесса


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Шаг 8: Запустите тестовый случай 1 - Город БЕЗ доступности (Париж)

Давайте проверим сценарий **отсутствия доступности**, запросив отели в Париже (в котором, согласно нашей симуляции, нет свободных номеров).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Шаг 9: Запуск тестового случая 2 - Город С Наличием (Стокгольм)

Теперь протестируем путь **наличия**, запросив отели в Стокгольме (в котором есть свободные номера в нашей симуляции).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Основные выводы и следующие шаги

### ✅ Что вы узнали:

1. **Шаблон WorkflowBuilder**
   - Используйте `.set_start_executor()` для определения точки входа
   - Используйте `.add_edge(from, to, condition=...)` для условной маршрутизации
   - Вызовите `.build()` для завершения создания рабочего процесса

2. **Условная маршрутизация**
   - Функции условий анализируют `AgentExecutorResponse`
   - Разбирайте структурированные выходные данные для принятия решений о маршрутизации
   - Возвращайте `True`, чтобы активировать связь, и `False`, чтобы пропустить её

3. **Интеграция инструментов**
   - Используйте `@ai_function` для преобразования функций Python в инструменты ИИ
   - Агенты автоматически вызывают инструменты, когда это необходимо
   - Инструменты возвращают JSON, который агенты могут анализировать

4. **Структурированные выходные данные**
   - Используйте модели Pydantic для безопасного извлечения данных по типу
   - Установите `response_format=MyModel` при создании агентов
   - Анализируйте ответы с помощью `Model.model_validate_json()`

5. **Пользовательские исполнители**
   - Используйте `@executor(id="...")` для создания компонентов рабочего процесса
   - Исполнители могут преобразовывать данные или выполнять побочные действия
   - Используйте `ctx.yield_output()` для генерации результатов рабочего процесса

### 🚀 Применение в реальном мире:

- **Бронирование путешествий**: Проверка доступности, предложение альтернатив, сравнение вариантов
- **Обслуживание клиентов**: Маршрутизация на основе типа проблемы, настроения, приоритета
- **Электронная коммерция**: Проверка наличия товара, предложение альтернатив, обработка заказов
- **Модерация контента**: Маршрутизация на основе уровня токсичности, пользовательских жалоб
- **Процессы утверждения**: Маршрутизация на основе суммы, роли пользователя, уровня риска
- **Многоэтапная обработка**: Маршрутизация на основе качества данных, их полноты

### 📚 Следующие шаги:

- Добавьте более сложные условия (несколько критериев)
- Реализуйте циклы с управлением состоянием рабочего процесса
- Добавьте подрабочие процессы для повторно используемых компонентов
- Интегрируйте реальные API (бронирование отелей, системы инвентаризации)
- Добавьте обработку ошибок и резервные пути
- Визуализируйте рабочие процессы с помощью встроенных инструментов визуализации



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
